# Setup

## config LLM

In [1]:
import autogen

config_list_gpt4 = autogen.config_list_from_dotenv(
    dotenv_file_path="../.env",
    model_api_key_map={
        "gpt-4o": {
            "api_key_env_var": "AZURE_OPENAI_API_KEY",
            "api_type": "azure",
            "api_version": "AZURE_OPENAI_API_VERSION",
            "base_url": "AZURE_OPENAI_ENDPOINT",
        },
    },
    filter_dict={
        "model": {
            "gpt-4o",
        }
    },
)

gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}
# print(gpt4_config)

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


## config MySQL

In [3]:
import os
from dotenv import load_dotenv
import mysql.connector

#load env variables
load_dotenv('../.env')

config_mysql = {
    'host': os.getenv('MYSQL_HOST'),
    'user': os.getenv('MYSQL_USR','luogang'),
    'password': os.getenv('MYSQL_PASWRD'),
    'database': 'tst1', #os.getenv('database','tst1'),
    # 'ssl_ca': os.getenv('ssl_ca')  # Path to the SSL CA certificate (optional)
}

conn = mysql.connector.connect(**config_mysql)

cusrmysql = conn.cursor()

# Build Agents

In [4]:

client = autogen.UserProxyAgent(
    name="client",
    system_message="想购买手机的顾客.经常会向手机销售咨询手机的价格和折扣优惠.",
    code_execution_config=False,
    human_input_mode="NEVER"
)
sqlexpert = autogen.AssistantAgent(
    name="SQL_expert",
    llm_config=gpt4_config,
    system_message="""你是一个SQL专家,熟悉MySQL数据库,能熟练地写出满足用户需求的SQL语句. 你必须一次性写出满足需求的, 完整的, 可执行的SQL语句. 由于错误的修复会耗费大量精力, 你要尽量保证SQL的执行成功与满足需求, 因此你若不清楚用户需求, 就需要不断地要求用户澄清, 完全了解用户需求后再需要结合表结构(也就是 table schema)与用户需求, 生成可执行的满足需求的SQL语句, 并调用 execute_sql() 去执行SQL语句. """,
)
sales = autogen.AssistantAgent(
    name="mobile_phone_saler",
    llm_config=gpt4_config,
    system_message="""你负责回答手机顾客问你的手机价格与折扣优惠相关的问题. 由于手机价格与优惠相关的问题需要准确, 真实的答案, 因此你每次回答前, 都要让SQL专家去生成SQL语句来查询数据库, 得到真实正确的数据后, 再基于这些数据作出回答.""",
)


In [5]:
from typing import Annotated, Dict

@sqlexpert.register_for_llm(description="Function for executing SQL query and returning a response. 用于执行sql语句的函数.")
@sales.register_for_execution()
def execute_sql(reflection: Annotated[str, "Think about what to do"], sql: Annotated[str, "SQL query"]) -> Annotated[Dict[str, str], "Dictionary with keys 'result' and 'error'"]:
    try:
        # Execute SQL queries
        cusrmysql.execute(sql)
        result = cusrmysql.fetchall()
    except Exception as e:
        return {
            "error": "SQL执行出错!",
            "error_msg": str(e),
        }

    return {"result": result}

In [6]:



groupchat = autogen.GroupChat(
    agents=[client, sales, sqlexpert], messages=[], max_round=50
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

# Start Chat

In [9]:
import logging
logging.basicConfig(level=logging.DEBUG)

client.initiate_chat(
    manager,
    message="""iPhone大概多少钱?""",
)

手机顾客 (to chat_manager):

iPhone大概多少钱?

--------------------------------------------------------------------------------


BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'messages[1].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[1].name', 'code': 'invalid_value'}}

In [11]:
u = autogen.UserProxyAgent(
    name="Admin",
    system_message="man like jokes",
    code_execution_config=False,
    human_input_mode="NEVER"
)
jkr = autogen.AssistantAgent(
    name="Joker",
    llm_config=gpt4_config,
    system_message="""man like making jokes""",
)
jk_grp = autogen.GroupChat(
    agents=[u, jkr], messages=[], max_round=50
)
manager = autogen.GroupChatManager(groupchat=jk_grp, llm_config=gpt4_config)
u.initiate_chat(
    manager,
    message="""tell me a joke""",
)

Admin (to chat_manager):

tell me a joke

--------------------------------------------------------------------------------



Next speaker: Joker

Joker (to chat_manager):

Why don't skeletons fight each other?

They don't have the guts!

--------------------------------------------------------------------------------



Next speaker: Admin

Admin (to chat_manager):



--------------------------------------------------------------------------------



Next speaker: Joker

Joker (to chat_manager):

What's orange and sounds like a parrot?

A carrot!

--------------------------------------------------------------------------------



Next speaker: Admin

Admin (to chat_manager):



--------------------------------------------------------------------------------


...


Next speaker: Admin

Admin (to chat_manager):



--------------------------------------------------------------------------------



Next speaker: Joker

Joker (to chat_manager):

It seems like you're enjoying this! Let me know if you want more jokes or anything else.

--------------------------------------------------------------------------------



Next speaker: Admin

Admin (to chat_manager):



--------------------------------------------------------------------------------



Next speaker: Admin

Admin (to chat_manager):



--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'tell me a joke', 'role': 'assistant', 'name': 'Admin'}, {'content': "Why don't skeletons fight each other?\n\nThey don't have the guts!", 'name': 'Joker', 'role': 'user'}, {'content': '', 'role': 'assistant', 'name': 'Admin'}, {'content': "What's orange and sounds like a parrot?\n\nA carrot!", 'name': 'Joker', 'role': 'user'}, {'content': '', 'role': 'assistant', 'name': 'Admin'}, {'content': 'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!', 'name': 'Joker', 'role': 'user'}, {'content': '', 'role': 'assistant', 'name': 'Admin'}, {'content': '', 'role': 'assistant', 'name': 'Admin'}, {'content': "It seems like you're really enjoying these! Want to hear another one?", 'name': 'Joker', 'role': 'user'}, {'content': 'Why did the bicycle fall over?\n\nBecause it was two-tired!', 'name': 'Joker', 'role': 'user'}, {'content': '', 'role': 'assistant', 'name': 'Admin'}, {'content': '', 'role': 'assistant', 'name